#### Введи имя модуля

In [7]:
true_mod_name = 'tst'
pyx_name = true_mod_name+'.pyx'
pxd_name = true_mod_name+'.pxd'

#### ниже строчки *'%%writefile tmp_module.pyx'* вставь Cython код

In [8]:
%%writefile $pyx_name

import sys
import os
sys.path.append(os.path.dirname(sys.path[0])+"\\invariants")
cimport tube as tb

cpdef void foo():
    cdef tb.Tube t = tb.Tube([1,2,3], [2,2,3])
    print(t)

Writing tst.pyx


In [6]:
%%writefile $pxd_name


cdef class InterpXY:
    cdef public double[:] xs, ys, ks, bs
    cdef int length, n
    cpdef double get_v(self, double x)
    cpdef double[:] get_vs(self, double[:] xs)
    cdef int set_n(self, double t)
    cdef void sync_ks_bs(self)

cdef class Tube:
    cdef InterpXY d, s, w, w_reverse
    cpdef double get_d(self, double x)
    cpdef double[:] get_ds(self, double[:] xs)
    cpdef double[:] get_xs(self)
    cpdef double[:] get_dsdx(self, double[:] xs)    
    cpdef double[:] get_W(self, double[:] xs)
    cpdef double[:] get_S(self, double[:] xs)
    cpdef double get_x2(self, double x1, double w)
    cpdef double get_W_between(self, double x1, double x2)

Writing tube.pxd


In [9]:
setup_string = \
f"""
from distutils.core import setup
from Cython.Build import cythonize
import sys
import os
sys.path.append(os.path.dirname(sys.path[0])+"\\invariants")
if __name__ == '__main__':
    setup(
        name='{true_mod_name}',
        ext_modules = cythonize("{pyx_name}", language="c++")
)
"""
setup_name = true_mod_name+'_setup.py'
with open(setup_name, mode='w') as f:
    f.write(setup_string)

In [10]:
!python $setup_name build_ext --inplace

Compiling tst.pyx because it changed.
[1/1] Cythonizing tst.pyx
running build_ext
building 'tst' extension
creating build
creating build\temp.win-amd64-3.7
creating build\temp.win-amd64-3.7\Release
C:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.15.26726\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\ProgramData\Anaconda3\include -IC:\ProgramData\Anaconda3\include "-IC:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.15.26726\ATLMFC\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Tools\MSVC\14.15.26726\include" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17134.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17134.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17134.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.17134.0\winrt" "-IC:\Program Files (x86)\Windows K

C:\ProgramData\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: D:\PyKas\jupyters\tst.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


### Чистим, переименовываем

In [11]:
import glob
import os
import shutil


try:
    f2 = glob.glob(true_mod_name+'.pyd')
    os.remove(f2[0])
except Exception as e:
    print(e)

f = glob.glob(true_mod_name+'*.pyd')
os.rename(f[0], true_mod_name+'.pyd')
# os.remove(pyx_name)
os.remove(true_mod_name+'.cpp')
os.remove(setup_name)
shutil.rmtree('build')

list index out of range


### Done!